In [1]:
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv

import moodle_admin.moodle_api as mp

In [2]:
courses = mp.call("core_course_get_courses")
df = pd.DataFrame(courses)

In [4]:
df

,id,shortname,categoryid,categorysortorder,fullname,displayname,idnumber,summary,summaryformat,format,...,groupmodeforce,defaultgroupingid,timecreated,timemodified,enablecompletion,completionnotify,lang,forcetheme,courseformatoptions,hiddensections
0,1,Aularagón,0,1,Aularagón,Aularagón,,,1,site,...,0,0,1488282562,1617710959,0,0,,,"[{'name': 'numsections', 'value': 0}]",NaN
1,3,LIBREOFFICE,43,150014,LibreOffice: La suite ofimática libre,LibreOffice: La suite ofimática libre,,<p>LibreOffice es un software desarrollado y d...,1,topics,...,0,0,1477304039,1552309228,1,0,,,"[{'name': 'hiddensections', 'value': 0}, {'nam...",0.0
2,4,ING-DUOLINGO,58,240008,Inglés Duolingo,Inglés Duolingo,,"<p>La educación personalizada es, cada día más...",1,topics,...,0,0,1488201052,1552047141,1,0,,,"[{'name': 'hiddensections', 'value': 0}, {'nam...",0.0
3,5,Radio,48,210003,La Radio en la Escuela,La Radio en la Escuela,,,1,topics,...,0,0,1487586139,1551439277,1,0,,,"[{'name': 'hiddensections', 'value': 0}, {'nam...",0.0
4,6,FLAUTA,48,210002,La enseñanza-aprendizaje de la flauta escolar,La enseñanza-aprendizaje de la flauta escolar,,<p></p><p>El presente curso va dirigido a doce...,1,topics,...,0,0,1489408019,1552047305,1,0,,,"[{'name': 'hiddensections', 'value': 0}, {'nam...",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
776,1085,BLANSERR,41,170002,BLANCO SERRANO,BLANCO SERRANO,,,1,topics,...,0,0,1614265721,1616331794,1,0,es,,"[{'name': 'hiddensections', 'value': 0}, {'nam...",0.0
777,1086,SIGAD-BASICO,2,280003,Curso Básico de SIGAD,Curso Básico de SIGAD,,,1,topics,...,0,0,1572263073,1572263073,1,0,,,"[{'name': 'hiddensections', 'value': 0}, {'nam...",0.0
778,1087,IMPR3D_TINK_CURA,48,210001,IMPRESIÓN BÁSICA EN 3D CON TINKERCAD Y CURA,IMPRESIÓN BÁSICA EN 3D CON TINKERCAD Y CURA,,"<p></p><h2 id=""objetivos""></h2><h2>Requisitos<...",1,topics,...,0,0,1617818214,1619086698,1,0,,,"[{'name': 'hiddensections', 'value': 0}, {'nam...",0.0
779,1088,ING-DUOLINGO_1,27,350001,Inglés Duolingo copia 1,Inglés Duolingo copia 1,,"<p>La educación personalizada es, cada día más...",1,topics,...,0,0,1488201052,1617966436,1,0,,,"[{'name': 'hiddensections', 'value': 0}, {'nam...",0.0


In [10]:
courses = mp.CourseList()

In [11]:
courses[0].